# OpenMendel: Iterative Hard Thresholding Tutorial

### Last update: 10/13/2018

### When to use Iterative Hard Thresholding

*Continuous* model selection is advantageous in situations where the multivariate nature of the regressors plays a significant role *together*. Iterative hard-thresholing (IHT) performs continuous model selection on a GWAS dataset $\mathbf{X}$ and continuous phenotype vector $\mathbf{y}$ by minimizing the residual sum of squares $f(\beta) = \frac{1}{2}||\mathbf{y} - \mathbf{X}\beta||^2$ subject to the constraint that $\beta$ is $k-$sparse. Parallel computing is offered through `q-`fold cross validation, and in the near future, dense (genotype matrix)-(dense vector) multiplication. 

### Appropriate Datasets and Example Inputs 

All genotype data **must** be stored in the [PLINK binary genotype format](https://www.cog-genomics.org/plink2/formats#bed), where the triplets `.bim`, `.bed` and `.fam` must all be present. Additional non-genetic covariates should be stored in a separate file (e.g. comma separated file). In this tutorial, we use "gwas 1 data" (github repo: [here](https://github.com/OpenMendel/MendelGWAS.jl/tree/master/docs)) to illustrate the functionalities of MendelIHT. This dataset has 2200 people and a modest 10000 simulated SNPs, with 2 SNPs (`rs1935681` and `rs2256412`) contributing to the response. One can obtain this dataset from the first example input of [MendelGWAS.jl](https://openmendel.github.io/MendelGWAS.jl/), or via option 24a of the free application [Mendel version 16](http://www.genetics.ucla.edu/software/mendel). 


### Missing Data

The current implementation of `MendelIHT` assumes there are no missing genotypes, since it uses linear algebra functions defined in [SnpArrays.jl](https://openmendel.github.io/SnpArrays.jl/latest/man/snparray/#linear-algebra-with-snparray). Therefore, you must first impute missing genotypes before you use MendelIHT. SnpArrays.jl offer a naive imputation strategy, but otherwise, our own software [option 23 of Mendel](http://www.genetics.ucla.edu/software/mendel) is a reasonbale choice. Open Mendel will soon provide a separate package `MendelImpute` containing new imputation strategies such as alternating least squares.  

### Cross Validation and Regularization paths

We usually have very little information on how many SNPs are affecting the phenotype. In a typical GWAS study, anywhere between 1 to thousands of SNPs could play a role. Thus ideally, we can test many different models to find the best one. MendelIHT provides 2 ways for one to perform this automatically: user specified regulartization paths, and $q-$fold [cross validation](https://en.wikipedia.org/wiki/Cross-validation_(statistics). Users should know that, in the first method, increasing the number of predictors will almost surely decrease the error, but as a result induces overfitting. Therefore, in most practicaly situations, it is highly recommended to combine this method with cross validation. $q-$fold cross validation divides samples into $q$ disjoint subsets, and minimizes the MSE of a model fitted on $q-1$ of those sets data, then verifies the model on the $qth$ samples. Each $q$ subsets are served as the test set exactly once. As a result, cross validation typically finds a model with a good balance of error and overfitting.  

### Julia version

Current code supports Julia version 0.6. It will not work in v0.7 or v1.0, but upgrade to v1.0 is very high on our TODO list. 

### Analysis keywords available to users 

| Keyword | Default Value | Allowed value | Description |
| --- | --- | --- | --- |
|`predictors` | 0 | Positive integer | Max number of non-zero entries of $\beta$ |
|`non_genetic_covariates` | "" | File name on disk | Delimited file containing the non-genetic covariates for each sample |
|`run_specific_paths` | false | boolean | Whether user wants to run a lot of different model sizes $k$ |
|`model_size_paths` | "" | Integers stored in string separated by ',' | Different model sizes users wish to run IHT for |
|`max_groups` (\*) | 1 | Integer | Total number of groups |
|`group_membership` (\*) | "" | File name on disk | File indicating group membership |
|`prior_weights` (\*) | "" | maf | How to scale predictors based on different weights |
|`run_cross_validation` (\**) | false | boolean | Whether the user wants to run cross-validation |
|`cv_fold` (\**) | 0 | Positive integer | Number of folds cross-validation should run |

+ (\*) Indicates experimental features. We currently have no theoretical guarantees on their performance, therefore illustrations of these functionalities are omitted from this tutorial. Users should tread carefully with these features. 
+ (\**) Indicates features that will soon be available
+ A list of OpenMendel keywords common to most analysis package can be found [here](https://openmendel.github.io/MendelBase.jl/#keywords-table)

# Example 1: Run IHT with Only Genotype Data

### Step 1: Preparing Input files

In Open Mendel, all analysis parameters are specified via the [Control file](https://openmendel.github.io/MendelBase.jl/#control-file). Genotype data must be inputted via the PLINK binary format. The most basic control file to run IHT looks like the following:

In [6]:
open("gwas 1 Control basic.txt") do file #change directory as appropriate
    for ln in eachline(file)
        println(ln)
    end
end

#
# Input and Output files.
#
plink_input_basename = gwas 1 data

#
# Analysis parameters for IHT option.
#
predictors = 2


### Step 2: Run MendelIHT

To run `MendelIHT`, execute the following in the Julia REPL or in this notebook:

In [9]:
using IHT
MendelIHT("gwas 1 Control basic.txt") # change directory as necessary

 
 
     Welcome to OpenMendel's
      IHT analysis option
        version 0.3.0
 
 
Reading the data.

The current working directory is "/Users/biona001/.julia/v0.6/IHT/test/MendelIHT_tutorial".

Keywords modified by the user:

  affected_designator = 2
  control_file = gwas 1 Control basic.txt
  pedigree_file = gwas 1 data.fam
  plink_input_basename = gwas 1 data
  predictors = 2
  snpdata_file = gwas 1 data.bed
  snpdefinition_file = gwas 1 data.bim
 


INFO: Reading in data
INFO: v1.0 BED file detected
INFO: Analyzing the data for model size k = 2


IHT results:

Compute time (sec):     1.892350432
Final loss:             1161.951128229144
Iterations:             12
Max number of groups:   1
Max predictors/group:   2
IHT estimated 2 nonzero coefficients.
2×3 DataFrames.DataFrame
│ Row │ Group │ Predictor │ Estimated_β │
├─────┼───────┼───────────┼─────────────┤
│ 1   │ 1     │ 3981      │ 0.149048    │
│ 2   │ 1     │ 7023      │ 0.272921    │

Intercept of model = 0.1405364595125695


### Step 3: Interpreting the results

Here the estimated model is the 3981th and 7023th predictor, corresponding to rs1935681 and rs2256412 in the `gwas 1 data.bim` file, which is the correct SNPs. The intercept of the model is given at the bottom of the table. Here the compute time is the time associated with computing the optimal $\beta$ only, so it does not include other necessary processes, such as importing the data. 

**Note:** the `affected_designator = 2` simply indicates that the pedigree is a Plink .fam file, which must always be the case for `MendelIHT` because this analysis option only accepts PLINK binary format as inputs. 

# Example 2: Including Non-Genetic Covariates

Non-genetic covariates must be stored in a comma demited file, with the same number of rows as the number of samples. The intercept term (i.e. grand mean) must also be included in the file. If the user does not specify a non-genetic covariate file, `MendelIHT` will by default include an intercept in the estimated model. 

### Step 1: Prepare Non-Genetic Covariate File

In this example, we generated one non-genetic covariate from a $N(0, 1)$ distribution. After including the grand mean, we saved the file in `gwas 1 noncov.txt` where the entries are separated by a tab. The first few lines of this file looks like the following:

In [13]:
f = open("gwas 1 noncov.txt")
lines = readlines(f)
for line in lines[1:10]   #print first 10 lines
    println(line)
end

1	-0.088704513339476
1	-0.9575873240069772
1	-0.9713258274139007
1	-0.9847900613424241
1	-0.5954781589540936
1	0.2124813875751884
1	2.28150775802523
1	1.7643235366779797
1	-0.3933262467789896
1	-0.1348394065324508


### Step 2: Prepare Corresponding Control File 

We need to tell MendelIHT that the covariates are separated by tabs. This can be achieved via the [MendelBase](https://openmendel.github.io/MendelBase.jl/) keyword `field_separator` as follows:

In [15]:
open("gwas 1 Control nongen.txt") do file #change directory as appropriate
    for ln in eachline(file)
        println(ln)
    end
end

#
# Input and Output files.
#
plink_input_basename = gwas 1 data
non_genetic_covariates = gwas 1 noncov.txt
field_separator = '	'
#
# Analysis parameters for IHT option.
#
predictors = 2


### Step 3: Run IHT 

In [16]:
using IHT
MendelIHT("gwas 1 Control nongen.txt") # change directory as necessary

 
 
     Welcome to OpenMendel's
      IHT analysis option
        version 0.3.0
 
 
Reading the data.

The current working directory is "/Users/biona001/.julia/v0.6/IHT/test/MendelIHT_tutorial".

Keywords modified by the user:

  affected_designator = 2
  control_file = gwas 1 Control nongen.txt
  field_separator = 	
  non_genetic_covariates = gwas 1 noncov.txt
  pedigree_file = gwas 1 data.fam
  plink_input_basename = gwas 1 data
  predictors = 2
  snpdata_file = gwas 1 data.bed
  snpdefinition_file = gwas 1 data.bim
 


INFO: Reading in data
INFO: v1.0 BED file detected
INFO: Analyzing the data for model size k = 2


IHT results:

Compute time (sec):     1.990755115
Final loss:             1161.9471299575046
Iterations:             12
Max number of groups:   1
Max predictors/group:   2
IHT estimated 2 nonzero coefficients.
2×3 DataFrames.DataFrame
│ Row │ Group │ Predictor │ Estimated_β │
├─────┼───────┼───────────┼─────────────┤
│ 1   │ 1     │ 3981      │ 0.149032    │
│ 2   │ 1     │ 7023      │ 0.272937    │

Intercept of model = 0.14055574614433


**Remark:** Observe that the overall error did not change by much, and the estimated coefficients and intercept remained roughly the same. 

# Example 3: Specify Regularization Paths

In the first 2 examples, IHT was ran on simulated data where the true model size $(k = 2)$ was known. In this example, users can run IHT on any number of model sets on a maximum of 8 parallel threads. Note that multithreading must be done in the terminal. Empirically, running on $n$ threads  achieves roughly $n/2$ fold speedup. 

### Step 1: Execute following line in terminal before starting notebook

In [ ]:
export JULIA_NUM_THREADS=8

### Step 2: Verify that notebook is indeed running with 8 threads:

In [1]:
Threads.nthreads()

8

### Step 3: Specify the model sizes via `model_size_paths`

The paths should be inside quotes and separated by comma. Each entry must be an integer. In this example, we tried to run IHT for model sizes $k = 1, 2, ..., 8$. The result is stored in the sparse matrix. 

In [1]:
open("gwas 1 Control userpath.txt") do file #change directory as appropriate
    for ln in eachline(file)
        println(ln)
    end
end

#
# Input and Output files.
#
plink_input_basename = gwas 1 data
#
# Analysis parameters for IHT option.
#
predictors = 2
model_size_paths = "1,2,3,4,5,6,7,8"


### Step 4: Run the Control File

In [1]:
using IHT
my_models = MendelIHT("gwas 1 Control userpath.txt")

 
 
     Welcome to OpenMendel's
      IHT analysis option
        version 0.3.0
 
 
Reading the data.

The current working directory is "/Users/biona001/.julia/v0.6/IHT/test/MendelIHT_tutorial".

Keywords modified by the user:

  affected_designator = 2
  control_file = gwas 1 Control userpath.txt
  model_sizes = 1,2,3,4,5,6,7,8
  pedigree_file = gwas 1 data.fam
  plink_input_basename = gwas 1 data
  predictors = 2
  snpdata_file = gwas 1 data.bed
  snpdefinition_file = gwas 1 data.bim
 


INFO: Reading in data
INFO: v1.0 BED file detected


10000×8 SparseMatrixCSC{Float64,Int64} with 36 stored entries:
  [7023 ,     1]  =  0.276207
  [3981 ,     2]  =  0.149048
  [7023 ,     2]  =  0.272921
  [3981 ,     3]  =  0.151119
  [5793 ,     3]  =  -0.0856503
  [7023 ,     3]  =  0.272561
  [3981 ,     4]  =  0.146525
  [4398 ,     4]  =  0.0836463
  [6611 ,     4]  =  0.0840132
  [7023 ,     4]  =  0.270291
  ⋮
  [5793 ,     7]  =  -0.087082
  [6611 ,     7]  =  0.0866336
  [7023 ,     7]  =  0.265817
  [898  ,     8]  =  -0.0766181
  [1880 ,     8]  =  -0.077302
  [3981 ,     8]  =  0.144214
  [4398 ,     8]  =  0.080625
  [5793 ,     8]  =  -0.0869424
  [6611 ,     8]  =  0.0857555
  [6627 ,     8]  =  -0.0737263
  [7023 ,     8]  =  0.264556

INFO: Running the following model sizes: [1, 2, 3, 4, 5, 6, 7, 8]


# Example 3.5: Comparison with regular GWAS

# Example 4: Cross Validation